In [ ]:
import numpy as np
import pandas as pd
import random
import pickle
import sys
from timeit import default_timer as timer
import datetime


# Importing Data and Labels

In [ ]:
chromosomeFile = "Data//matchesList.txt"
labelsFile = "Data//labelList.txt"

with open(chromosomeFile, 'rb') as f:
    matchList = pickle.load(f)

with open(labelsFile, 'rb') as f:
    labels = pickle.load(f)

for x in matchList:
    del x[8]
    del x[0]

# Importing Data and Labels

In [ ]:
def generatePopulation(size):
    population = []
    for i in range(size):
        temp = []
        for j in range(0,14):
            temp.append(random.randint(0,256))
        population.append(temp)
    return population

In [ ]:
def getExpr():
    expr = ["<expr> + <expr>", "<expr> - <expr>", "<expr> * <expr>", "<expr> / <expr>"]
    for i in range(0,7):
            expr.append("t1Stats[" + str(i) + "]")
    return expr

In [ ]:
def formGrammar(chromosome):
    ex = "<expr>"
    grammar = [ex]
    expr = getExpr()
    for c in range(0,len(chromosome)):
        for i in range(0,len(grammar)):
            if(grammar[i]==ex):
                addToGrammar = expr[chromosome[c]%len(expr)].split(" ")
                for x in addToGrammar:
                    grammar.insert(i+1,x)
                del grammar[i]
                break
    #t1 = grammar.copy()
    #t2 = grammar.copy()
    #for i in t2:
        #t2[t2.index(i)] = i.replace("t1","t2")
    return grammar

In [ ]:
def formString(grammar):
    string = ""
    for i in grammar:
        string += i + " "
    return string

In [ ]:
def evalString(grammarString, eMatchList, eLabels):
    tempMatchList = eMatchList.copy()
    fitness = 0
    t1GrammarString = grammarString
    t2GrammarString = grammarString.replace("t1","t2")
    labels = []
    for i in range(len(eMatchList)):
        try:
            t1Stats = eMatchList.copy()[i][:7]
            t2Stats = eMatchList.copy()[i][7:]
            t1v = eval(t1GrammarString)
            t2v = eval(t2GrammarString)
            finalVal = t1v - t2v
            if(finalVal>0.25):
                labels.append("H")
            elif(finalVal<-0.25):
                labels.append("A")
            else:
                labels.append("D")    
            #print(finalVal)
            #print(t1Stats,"\n",t2Stats,"\n",t1GrammarString,"\n",t2GrammarString,"\n",t1v,"\n",t2v)
        except:
            #print("Error", grammarString, sys.exc_info()[0])
            return -1
    index = 53
    for i in range(0,len(labels)):
        if(labels[i]==eLabels[i]):
            fitness+=1
    fitness = fitness/len(labels)
    return fitness

In [ ]:
workingChromosome = [14, 240, 188, 104, 14, 163, 138, 196, 7, 205, 192, 36, 111, 38]
expr = getExpr()
workingGrammar = formGrammar(workingChromosome)
workingGrammarString = formString(workingGrammar)
print(workingChromosome)
print(workingGrammarString)

In [ ]:
brokenChromosome = [44, 1, 242, 192, 239, 44, 154, 219, 34, 66, 128, 75, 107, 98]
expr = getExpr()
brokenGrammar = formGrammar(brokenChromosome)
brokenGrammarString = formString(brokenGrammar)
print(brokenChromosome)
print(brokenGrammarString)

In [ ]:
evalString(workingGrammarString, matchList, labels)

In [ ]:
evalString(brokenGrammarString, matchList, labels)

In [ ]:
def tournamentSelection(population,size, sMatchList, sLabels):
    tournament = []
    for i in range(size):
        tournament.append(population[random.randint(0,len(population)-1)])
    
    selectedEval = []
    expr = getExpr()
    for i in range(0,len(tournament)):
        selectedEval.append(evalString(formString(formGrammar(tournament[i])), sMatchList, sLabels))
    best = selectedEval.index(max(selectedEval))
    return tournament[best]

In [ ]:
def crossover(parent1, parent2,k):
    child = parent1[0:k] + parent2[k:]
    return list(child)

In [ ]:
def mutation(child, mutationCount):
    for i in range(0,mutationCount):
        child[random.randint(0,len(child)-1)] = random.randint(0,256)
    return child

In [ ]:
def evolve(populationSize, tournamentSize, mutationCount, generations, eMatchList, eLabels):
    population = generatePopulation(populationSize)
    stages = [0]*generations
    for i in range(generations):
        #Get Fitness Of Each Chromosome
        fitness = []
        expr = getExpr()
        for j in range(len(population)):
            grammar = formGrammar(population[j])
            grammarString = formString(grammar)
            fitness.append(evalString(grammarString, eMatchList, eLabels))
        child1, child2 = tournamentSelection(population, tournamentSize, eMatchList, eLabels), tournamentSelection(population, tournamentSize, eMatchList, eLabels)
        child = crossover(child1, child2, random.randint(0,len(child1)-1))
        child = mutation(child, mutationCount)
        newFitnessGrammar = formGrammar(child)
        newFitnessGrammarString = formString(newFitnessGrammar)
        newFitness = evalString(newFitnessGrammarString, eMatchList, eLabels)
        if(newFitness > min(fitness)):
            population[fitness.index(min(fitness))] = child
        stages[i] = [i,max(fitness),sum(fitness)/len(fitness),fitness.index(max(fitness))]
    text = "Population Size: " + str(populationSize) + " Tournament Size: " + str(tournamentSize) + " Generations: " + str(generations) + " Mutation Count: " + str(mutationCount) + " Best: " + str(max(fitness)) + ", Average: " + str(sum(fitness)/len(fitness))
    return [text,population[fitness.index(max(fitness))], stages]
        #print(child1,child2)
        
        #print(fitness) 
        #

In [ ]:
def run(p,t,m,g,rMatchList,rLabels):
    start = timer()
    populationSize = p
    tournamentSize = t
    mutationCount = m
    generations = g
    infoText, chromosome, stages = evolve(populationSize, tournamentSize, mutationCount, generations, rMatchList, rLabels)
    infoText = infoText + ", Time: " + str(timer()-start)
    infoText = infoText + ", Date: " + str(datetime.datetime.now())

    with open("Results//GRE//testsGRE.txt","a") as f:
        f.write(str(infoText) + "\n")
    with open("Results//GRE//chromosomeGRE.txt","a") as f:
        f.write(str(chromosome) + "\n")

    date = str(datetime.datetime.now())
    date = date.replace(" ","")[5:len(date)-8]
    date = "test" + date + ".txt"
    date = date.replace(":","_")
    date = date[:9] + "_" + date[9:]
    date = date[:len(date)-4] + "-GRE" + date[len(date)-4:]

    with open("Results//GRE//Individual//"+str(date),"w") as f:
        f.write("Generation,Best,Average,Index\n")
        for i in stages:
            f.write(str(i).replace("[","").replace("]","") + "\n")
    
    return "done"

In [ ]:
#pop = 10
#tour = 3
#gen = 100
#mut = 3
#print(run(pop,tour,mut,gen,matchList,labels))
populationSizes = [10,50,100,500]
tournamentSizes = [3,6]
mutationCounts = [1,2,3]
genarations = [10,100,250,500,1000]
for g in genarations:
    for m in mutationCounts:
        for t in tournamentSizes:
            for p in populationSizes:
                run(p, t, m, g, matchList, labels)
                pass

In [ ]:
#Started at 1:28
#Interupted at 1:43am next day